<img src="../../docs/images/DSPy8.png" alt="DSPy7 图片" height="150"/>## **DSPy 断言**: 对基础计算约束的断言### **LongFormQA**: 生成长篇长度的回答问题的响应

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/examples/longformqa/longformqa_assertions.ipynb)这个笔记本是在我们之前的教程中介绍的**DSPy**框架的基本概念基础上构建的（请参阅[intro.ipynb](./intro.ipynb)进行复习）。DSPy提供了一种新颖的以编程为中心的方法来利用语言和检索模型。它提供了一种独特的提示、推理、微调和工具增强的组合，所有这些都包含在一个简约的Python语法下。在DSPy的这一进展中，我们介绍了**Assertions**，这是一个具有在DSPy程序中声明计算约束能力的特性。这使程序员能够为有效输出指定自然语言规则，引导语言模型调用在编译和推理阶段的行为。我们的方法利用了Python风格的断言，同时将回溯逻辑融合在一起，以确保自主的自我校正和语言模型调用的精炼。通过考虑过去的输出并向前传递相关反馈和自我校正的指导，这个特性在DSPy中提供了一个重大的飞跃，增强了对程序行为的控制。这个笔记本演示了在特定下游示例中断言的实用性，将多跳问题回答任务从[intro.ipynb](./intro.ipynb)扩展到长篇段落生成，并引用来回答问题。我们展示了集成断言以确保引文以预定义格式包含在内以及生成文本与引用参考的忠实性的性能优势。

### 0] 设置让我们从设置开始。下面的代码片段将检索任务的缓存请求。

In [ ]:
!git clone https://huggingface.co/arnavs11/DSPy_LongFormQA%cd DSPy_LongFormQA!git checkout master%cd ..import osrepo_clone_path = '/content/DSPy_LongFormQA'# 检查'/content'是否可写if not os.access('/content', os.W_OK):    # 如果'/content'不可写，选择一个替代目录    # 例如：使用相对于当前工作目录的目录    repo_clone_path = os.path.join(os.getcwd(), 'DSPy_LongFormQA')# 为此笔记本设置缓存os.environ["DSP_NOTEBOOK_CACHEDIR"] = repo_clone_path

我们还将安装**DSPy**，如果它尚未安装。

In [ ]:
%load_ext autoreload%autoreload 2import sysimport osimport regex as retry: # 当在 Google Colab 上时，克隆笔记本以便下载缓存。    import google.colab    repo_path = 'dspy'        !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_pathexcept:    repo_path = '.'if repo_path not in sys.path:    sys.path.append(repo_path)import pkg_resources # 如果未安装该包，则安装该包if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:    !pip install -U pip    !pip install dspy-ai    !pip install openai~=0.28.1    !pip install -e $repo_pathimport dspyfrom dspy.predict import Retryfrom dspy.datasets import HotPotQAfrom dspy.teleprompt import BootstrapFewShotWithRandomSearchfrom dsp.utils import EM, normalize_textfrom dspy.primitives.assertions import assert_transform_module, backtrack_handler%cd dspy/examples/longformqafrom utils import extract_text_by_citation, correct_citation_format, has_citations, citations_check

### 1] 入门指南我们将从设置语言模型（LM）和检索模型（RM）开始。**DSPy**支持多种API和本地模型。在这个笔记本中，我们将使用GPT-3.5（`gpt-3.5-turbo`）和检索器`ColBERTv2`。为了简化操作，我们已经设置了一个ColBERTv2服务器，托管了一个维基百科2017年“摘要”搜索索引（即包含来自这个[2017年转储](https://hotpotqa.github.io/wiki-readme.html)的每篇文章的第一段），所以您不需要担心设置一个！而且是免费的。我们将**DSPy**配置为默认使用turbo LM和ColBERTv2检索器（在维基百科2017年摘要上）。如果需要，这可以被覆盖为程序的本地部分。

In [ ]:
# 创建一个 ColBERTv2 模型，使用指定的 URL 连接到 wiki17_abstracts 数据集colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')# 配置 DeepSpeed 设置，指定使用上面创建的 ColBERTv2 模型dspy.settings.configure(rm=colbertv2_wiki17_abstracts)# 创建一个 OpenAI 模型，使用 'gpt-3.5-turbo-0613' 模型和最大 token 数为 500turbo = dspy.OpenAI(model='gpt-3.5-turbo-0613', max_tokens=500)# 配置 DeepSpeed 设置，指定使用上面创建的 OpenAI 模型，关闭跟踪信息，设置温度为 0.7dspy.settings.configure(lm=turbo, trace=[], temperature=0.7)

### 2] 数据集现在，让我们为我们的任务加载HotPotQA多跳数据集中的一个样本。

In [ ]:
# 导入HotPotQA类from dataset import HotPotQA# 创建HotPotQA数据集对象，指定训练集大小为300，使用种子1进行训练集划分，验证集大小为300，使用种子2023进行验证集划分，测试集大小为0，保留详细信息dataset = HotPotQA(train_seed=1, train_size=300, eval_seed=2023, dev_size=300, test_size=0, keep_details=True)# 从训练集中提取问题作为输入，构建训练集trainset = [x.with_inputs('question') for x in dataset.train]# 从验证集中提取问题作为输入，构建验证集devset = [x.with_inputs('question') for x in dataset.dev]

我们刚刚加载了`trainset`（300个示例）和`devset`（300个示例）。我们的**训练集**中的每个示例只包含一个**问题**，其对应的（人工标注的）**答案**和**黄金标题**。这些黄金标题代表包含回答问题所需支持事实的相关维基百科文章的标题。在加载数据集后，我们对每个示例应用了`x.with_inputs('question')`，告诉**DSPy**我们每个示例中的输入字段将只是`question`。任何其他字段都是未提供给系统的标签。现在，让我们看一些数据示例。

In [ ]:
train_example = trainset[0]  # 从训练集中获取第一个样本print(f"Question: {train_example.question}")  # 打印问题print(f"Answer: {train_example.answer}")  # 打印答案print(f"Relevant Wikipedia Titles: {train_example.gold_titles}")  # 打印相关的维基百科标题

In [ ]:
# 选择第18个样本作为开发集示例dev_example = devset[18]# 打印问题print(f"Question: {dev_example.question}")# 打印答案print(f"Answer: {dev_example.answer}")# 打印相关的维基百科标题print(f"Relevant Wikipedia Titles: {dev_example.gold_titles}")

### 3] 具有引用的长篇问答让我们为这个任务定义我们的第一个完整程序。我们扩展了`Multi-Hop QA`程序，将答案生成的焦点从1-5个词的简短短语转移到包含引用的全面段落。`LongFormQA`模块反映了在查询生成、段落检索和上下文组装中的迭代多跳生成过程。然后，`GenerateCitedParagraph`层将上下文状态与问题一起使用，生成一个带有相关引用上下文的段落。

使用这个程序，我们的目标是生成符合以下准则的段落：1. 段落中的每1-2句话后面都跟着按照预期格式的引用 **"{text}... [source_num]."**2. 每个引用前的文本段落都忠实于所引用的源文段。

In [ ]:
from dsp.utils import deduplicateclass GenerateSearchQuery(dspy.Signature):    """编写一个简单的搜索查询，以帮助回答一个复杂的问题。"""    context = dspy.InputField(desc="可能包含相关事实")    question = dspy.InputField()    query = dspy.OutputField()class GenerateCitedParagraph(dspy.Signature):    """生成带引用的段落。"""    context = dspy.InputField(desc="可能包含相关事实")    question = dspy.InputField()    paragraph = dspy.OutputField(desc="包含引用")class LongFormQA(dspy.Module):    def __init__(self, passages_per_hop=3, max_hops=2):        super().__init__()        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]        self.retrieve = dspy.Retrieve(k=passages_per_hop)        self.generate_cited_paragraph = dspy.ChainOfThought(GenerateCitedParagraph)        self.max_hops = max_hops        def forward(self, question):        context = []        for hop in range(self.max_hops):            query = self.generate_query[hop](context=context, question=question).query            passages = self.retrieve(query).passages            context = deduplicate(context + passages)        pred = self.generate_cited_paragraph(context=context, question=question)        pred = dspy.Prediction(context=context, paragraph=pred.paragraph)        return pred

### 4] 评估我们现在定义我们的评估指标，**内在**和**外在**质量检查：#### 内在指标：通过内部计算约束是目标**忠实度（每个引用）**：为了验证生成文本中每个引用的准确性，我们利用另一个**DSPy**程序：`ChainOfThought`的`CheckCitationFaithfulness`。该模块获取每个引用之前的文本段和其相应的上下文段，确定文本是否准确反映了上下文的事实。这个验证过程涉及对每个引用的语言模型调用，确保生成段落中的每个参考事实上与其参考来源一致。

In [ ]:
class CheckCitationFaithfulness(dspy.Signature):    """验证文本是否基于提供的上下文。"""    context = dspy.InputField(desc="可能包含相关事实")    text = dspy.InputField(desc="1到2句之间的文本")    faithfulness = dspy.OutputField(desc="布尔值，指示文本是否忠实于上下文")def citation_faithfulness(example, pred, trace):    paragraph, context = pred.paragraph, pred.context    citation_dict = extract_text_by_citation(paragraph)    if not citation_dict:        return False, None    context_dict = {str(i): context[i].split(' | ')[1] for i in range(len(context))}    faithfulness_results = []    unfaithful_citations = []    check_citation_faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)    for citation_num, texts in citation_dict.items():        if citation_num not in context_dict:            continue        current_context = context_dict[citation_num]        for text in texts:            try:                result = check_citation_faithfulness(context=current_context, text=text)                is_faithful = result.faithfulness.lower() == 'true'                faithfulness_results.append(is_faithful)                if not is_faithful:                    unfaithful_citations.append({'paragraph': paragraph, 'text': text, 'context': current_context})            except ValueError as e:                faithfulness_results.append(False)                unfaithful_citations.append({'paragraph': paragraph, 'text': text, 'error': str(e)})    final_faithfulness = all(faithfulness_results)    if not faithfulness_results:        return False, None    return final_faithfulness, unfaithful_citations

#### 外部度量指标：评估生成输出在下游任务中的整体质量和有效性：- **引文精度**：衡量在数据点的所有引用标题中，生成段落中引用的“gold titles”的比例。- **引文召回**：衡量在数据点的所有“gold titles”中，生成段落中引用的“gold titles”的比例。- **答案包含**：评估生成的带引文段落是否准确地包含了数据点的“gold”答案。

In [ ]:
import re# 从段落中提取引用的标题def extract_cited_titles_from_paragraph(paragraph, context):    # 使用正则表达式找到段落中引用标题的索引    cited_indices = [int(m.group(1)) for m in re.finditer(r'\[(\d+)\]\.', paragraph)]    # 筛选出不超过上下文长度的索引    cited_indices = [index - 1 for index in cited_indices if index <= len(context)]    # 根据索引获取引用的标题    cited_titles = [context[index].split(' | ')[0] for index in cited_indices]    return cited_titles# 计算召回率def calculate_recall(example, pred, trace=None):    gold_titles = set(example['gold_titles'])    found_cited_titles = set(extract_cited_titles_from_paragraph(pred.paragraph, pred.context))    intersection = gold_titles.intersection(found_cited_titles)    recall = len(intersection) / len(gold_titles) if gold_titles else 0    return recall# 计算精确率def calculate_precision(example, pred, trace=None):    gold_titles = set(example['gold_titles'])    found_cited_titles = set(extract_cited_titles_from_paragraph(pred.paragraph, pred.context))    intersection = gold_titles.intersection(found_cited_titles)    precision = len(intersection) / len(found_cited_titles) if found_cited_titles else 0    return precision# 判断回答是否正确def answer_correctness(example, pred, trace=None):    assert hasattr(example, 'answer'), "Example does not have 'answer'."    normalized_context = normalize_text(pred.paragraph)    if isinstance(example.answer, str):        gold_answers = [example.answer]    elif isinstance(example.answer, list):        gold_answers = example.answer    else:        raise ValueError("'example.answer' is not string or list.")    return 1 if any(normalize_text(answer) in normalized_context for answer in gold_answers) else 0

我们现在在开发集上评估我们的程序的这些指标。

In [ ]:
def evaluate(module):    correctness_values = []  # 存储正确性值    recall_values = []  # 存储召回率值    precision_values = []  # 存储精确率值    citation_faithfulness_values = []  # 存储引文忠实度值    for i in range(len(devset)):  # 遍历开发集中的例子        example = devset[i]  # 获取当前例子        try:            pred = module(question=example.question)  # 预测答案            correctness_values.append(answer_correctness(example, pred))  # 计算正确性值并添加到列表中            citation_faithfulness_score, _ = citation_faithfulness(None, pred, None)  # 计算引文忠实度分数            citation_faithfulness_values.append(citation_faithfulness_score)  # 将引文忠实度值添加到列表中            recall = calculate_recall(example, pred)  # 计算召回率            precision = calculate_precision(example, pred)  # 计算精确率            recall_values.append(recall)  # 将召回率值添加到列表中            precision_values.append(precision)  # 将精确率值添加到列表中        except Exception as e:            print(f"Failed generation with error: {e}")  # 捕获异常并打印错误信息    average_correctness = sum(correctness_values) / len(devset) if correctness_values else 0  # 计算平均正确性    average_recall = sum(recall_values) / len(devset) if recall_values else 0  # 计算平均召回率    average_precision = sum(precision_values) / len(devset) if precision_values else 0  # 计算平均精确率    average_citation_faithfulness = sum(citation_faithfulness_values) / len(devset) if citation_faithfulness_values else 0  # 计算平均引文忠实度    print(f"Average Correctness: {average_correctness}")  # 打印平均正确性    print(f"Average Recall: {average_recall}")  # 打印平均召回率    print(f"Average Precision: {average_precision}")  # 打印平均精确率    print(f"Average Citation Faithfulness: {average_citation_faithfulness}")  # 打印平均引文忠实度

In [ ]:
longformqa = LongFormQA()  # 创建一个 LongFormQA 类的实例对象evaluate(longformqa)  # 对该实例对象进行评估

让我们来看一个段落生成的示例：

In [ ]:
# 获取第28个样本的问题question = devset[28].question# 使用 longformqa 函数对问题进行预测pred = longformqa(question)# 计算引文忠实度分数citation_faithfulness_score, _ = citation_faithfulness(None, pred, None)# 打印问题内容print(f"Question: {question}")# 打印预测的段落print(f"Predicted Paragraph: {pred.paragraph}")# 打印引文忠实度分数print(f"Citation Faithfulness: {citation_faithfulness_score}")

我们可以看到生成的段落并没有按照预期正确地包含引用。在某些情况下，它遵循了一个不正确的引用格式，即在句子末尾的句点之前没有保留引用的来源。在其他情况下，它没有像预期那样为每1-2个句子提供引用。此外，我们发现并非所有包含的引用都忠实于其前文（这可能是由于引用参考文献的格式不正确导致的）。

### 5] 引入断言：LongFormQAWithAssertions为了纠正这些错误，我们引入**断言**来在程序中施加明确的计算约束。DSPy 提供了两种关键机制用于**断言**：- **`dspy.Assert`**：这要求程序必须满足给定的断言，否则会引发异常。在强制执行程序内不可协商的约束时，这一点非常重要。- **`dspy.Suggest`**：与`Assert`不同，`Suggest`更加灵活。它鼓励程序满足断言，但即使断言不被满足，程序也可以继续执行。这对于引导程序朝着期望的结果发展而不会因为非关键问题而停止执行非常有用。由于我们的目标确实是根据定义的指标评估程序，让我们利用`dspy.Suggest`断言。`dspy.Suggest`的语法如下：```pythondspy.Suggest(validation_function(model_outputs): bool, instruction_message: str)```让我们添加断言以遵守上述定义的计算约束。

In [ ]:
class LongFormQAWithAssertions(dspy.Module):    def __init__(self, passages_per_hop=3, max_hops=2):        super().__init__()        # 生成查询问题的模块链        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]        # 检索模块，每次检索返回 passages_per_hop 个段落        self.retrieve = dspy.Retrieve(k=passages_per_hop)        # 生成引用段落的模块        self.generate_cited_paragraph = dspy.ChainOfThought(GenerateCitedParagraph)        self.max_hops = max_hops        def forward(self, question):        context = []        for hop in range(self.max_hops):            # 生成查询问题            query = self.generate_query[hop](context=context, question=question).query            # 检索相关段落            passages = self.retrieve(query).passages            # 去重后更新上下文            context = deduplicate(context + passages)        # 生成引用段落        pred = self.generate_cited_paragraph(context=context, question=question)        pred = dspy.Prediction(context=context, paragraph=pred.paragraph)        # 检查引文准确性        dspy.Suggest(citations_check(pred.paragraph), f"确保每1-2句话都有引文。如果有1-2句没有引文，请以 '文本... [x].' 的格式添加引文。", target_module=GenerateCitedParagraph)        _, unfaithful_outputs = citation_faithfulness(None, pred, None)        if unfaithful_outputs:            unfaithful_pairs = [(output['text'], output['context']) for output in unfaithful_outputs]            for _, context in unfaithful_pairs:                # 如果存在不忠实的输出，提出建议                dspy.Suggest(len(unfaithful_pairs) == 0, f"确保你的输出基于以下上下文: '{context}'.", target_module=GenerateCitedParagraph)        else:            return pred        return pred

我们包含了简单重申我们的计算约束的断言，并且现在允许`LongFormQA`程序在幕后执行并遵守这些准则。由于我们希望将这些断言强加于段落生成中，我们可以传递`GenerateCitedParagraph`签名以指示`target_module`用于断言处理的识别。在第一个**断言**中，我们验证输出段落以确保每1-2个句子中都包含引用。如果此验证返回False，则激活断言回溯逻辑并提供反馈指令：**"确保每1-2个句子以'text... [x].'格式包含引用。"**在第二个**断言**中，我们现在利用`CheckCitationFaithfulness`程序验证每个引用参考的准确性，循环遍历在生成的段落中标记的文本片段。在存在不忠实引用的情况下，它会发送反馈指令以及上下文作为：**"确保您的输出与此上下文一致：'{context}'."** 这确保了断言回溯具有所需的相关信息和特定上下文。

现在让我们在开发集上评估我们的 `LongFormQAWithAssertions` 程序。请注意，这需要使用 `Retry` 模块对模块进行包装，该模块处理回溯逻辑。然后将这个包装模块传递给 `assert_transform_module` 函数，以准备和执行回溯逻辑。这与配置回溯逻辑的 `backtrack_handler` 一起传递，以考虑程序中传递给 `dspy.Suggest` 语句的反馈消息。

In [ ]:
# 定义一个长形式问答模型，并添加断言转换模块longformqa_with_assertions = assert_transform_module(LongFormQAWithAssertions().map_named_predictors(Retry), backtrack_handler) # 评估长形式问答模型evaluate(longformqa_with_assertions)

让我们使用`LongFormQAWithAssertions`程序来查看上面相同的示例：

In [ ]:
# 获取第28个样本的问题question = devset[28].question# 使用 longformqa_with_assertions 函数对问题进行预测pred = longformqa_with_assertions(question)# 使用 citation_faithfulness 函数计算引文忠实度得分citation_faithfulness_score, _ = citation_faithfulness(None, pred, None)# 打印问题print(f"Question: {question}")# 打印预测的段落print(f"Predicted Paragraph: {pred.paragraph}")# 打印引文忠实度得分print(f"Citation Faithfulness: {citation_faithfulness_score}")

我们现在看到，确实满足了计算约束。每1-2句话都包含一次引用，从我们的引文忠实性检查中，我们看到每个参考文献也忠实于其前文。

### 6] 使用断言进行编译我们还可以利用**DSPy**的高级编译功能来增强我们程序的性能。为此，我们利用`BootstrapFewShotWithRandomSearch`提示器，该提示器自动整合少样本演示，并对候选集进行随机搜索，以输出最佳编译程序。我们将此评估为`answer_correctness`指标，因为我们的最终目标确实是从段落中生成正确答案以回答`HotPotQA`的问题，旨在优化内在和外在指标。让我们首先在LongFormQA程序上进行评估：

In [ ]:
# 创建一个 LongFormQA 实例longformqa = LongFormQA()# 创建一个 BootstrapFewShotWithRandomSearch 实例，指定评估指标为 answer_correctness，最大 bootstrap 次数为 2，候选程序数量为 6teleprompter = BootstrapFewShotWithRandomSearch(metric=answer_correctness, max_bootstrapped_demos=2, num_candidate_programs=6)# 使用 BootstrapFewShotWithRandomSearch 实例对 LongFormQA 进行编译，指定学生和老师都为 longformqa，训练集为 trainset，验证集为 devset 的前 100 个样本cited_longformqa = teleprompter.compile(student=longformqa, teacher=longformqa, trainset=trainset, valset=devset[:100])# 评估 cited_longformqaevaluate(cited_longformqa)

让我们使用断言来评估这个过程。**注意** 这里的流程在编译时使用**断言**，通过`answer_correctness`指标为提示器提供正确的引导示例，然后用这些正确的示例来“教导”学生。这通过将`LongFormQA()`作为学生传递，将`LongFormQAWithAssertions()`作为老师来表示。

In [ ]:
# 创建一个 LongFormQA 实例longformqa = LongFormQA()# 创建一个 BootstrapFewShotWithRandomSearch 实例，指定评估指标为 answer_correctness，最大引导演示数量为2，候选程序数量为6teleprompter = BootstrapFewShotWithRandomSearch(metric=answer_correctness, max_bootstrapped_demos=2, num_candidate_programs=6)# 使用 teleprompter 编译学生模型 longformqa，教师模型为 LongFormQAWithAssertions().map_named_predictors(Retry) 和 backtrack_handler，训练集为 trainset，验证集为 devset 的前100个样本cited_longformqa_teacher = teleprompter.compile(student=longformqa, teacher=assert_transform_module(LongFormQAWithAssertions().map_named_predictors(Retry), backtrack_handler), trainset=trainset, valset=devset[:100])# 评估 cited_longformqa_teacher 模型evaluate(cited_longformqa_teacher)

**注意** 另一方面，此流程将教师和学生都设置为 `LongFormQAWithAssertions()`，以确保教师正确指导学生使用正确的引导示例，并且学生有机会通过**断言**自我纠正任何仍被视为不正确的示例。

In [ ]:
longformqa = LongFormQA()  # 创建一个 LongFormQA 实例teleprompter = BootstrapFewShotWithRandomSearch(metric=answer_correctness, max_bootstrapped_demos=2, num_candidate_programs=6)  # 创建一个 BootstrapFewShotWithRandomSearch 实例，设置评估指标为 answer_correctness，最大引导演示数为2，候选程序数为6cited_longformqa_student_teacher = teleprompter.compile(    student=assert_transform_module(LongFormQAWithAssertions().map_named_predictors(Retry), backtrack_handler),  # 编译学生模型，添加断言和重试机制    teacher=assert_transform_module(LongFormQAWithAssertions().map_named_predictors(Retry), backtrack_handler),  # 编译教师模型，添加断言和重试机制    trainset=trainset,  # 训练集    valset=devset[:100]  # 验证集，取前100个样本)evaluate(cited_longformqa_student_teacher)  # 评估模型